In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df=pd.read_csv('train.csv')
df.isna().sum()

,0
Item_Identifier,0
Item_Weight,1463
Item_Fat_Content,0
Item_Visibility,0
Item_Type,0
Item_MRP,0
Outlet_Identifier,0
Outlet_Establishment_Year,0
Outlet_Size,2410
Outlet_Location_Type,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df.Item_Fat_Content= df.Item_Fat_Content.replace('lf', 'Low Fat')
df.Item_Fat_Content= df.Item_Fat_Content.replace('reg', 'Regular')
df.Item_Fat_Content= df.Item_Fat_Content.replace('low fat', 'Low Fat')
df.Item_Fat_Content= df.Item_Fat_Content.replace('LF', 'Low Fat')

In [ ]:
df.Item_Fat_Content = le.fit_transform(df.Item_Fat_Content)

In [ ]:
# Print unique values before mapping
print("Unique values for 'Outlet_Size':", df['Outlet_Size'].unique())

# Replace "missing" with NaN (though "missing" is not present, keeping this for general use)
df = df.replace("missing", np.nan)
df.Outlet_Size.fillna("Small",inplace=True)

# Correct mapping dictionary syntax
Sizenum = {
    "High": 3,
    "Medium": 2,
    "Small": 1
}

# Replace categorical values with numeric values using the mapping
df['Outlet_Size'] = df['Outlet_Size'].map(Sizenum)



Unique values for 'Outlet_Size': ['Medium' nan 'High' 'Small']


In [ ]:
# Print unique values before mapping
print("Unique values for 'Outlet_Location_Type':", df['Outlet_Location_Type'].unique())

# Replace "missing" with NaN (though "missing" is not present, keeping this for general use)
df = df.replace("missing", np.nan)

# Correct mapping dictionary syntax for 'Outlet_Location_Type'
LocationNum = {
    "Tier 1": 3,
    "Tier 2": 2,
    "Tier 3": 1
}

# Replace categorical values with numeric values using the mapping
df['Outlet_Location_Type'] = df['Outlet_Location_Type'].map(LocationNum)

# Print the updated column to verify changes
print("Updated 'Outlet_Location_Type' column with numeric values:")
print(df['Outlet_Location_Type'].unique())

# Check value counts to ensure mapping was applied correctly
print("Value counts in 'Outlet_Location_Type' column:")
print(df['Outlet_Location_Type'].value_counts(dropna=False))

Unique values for 'Outlet_Location_Type': ['Tier 1' 'Tier 3' 'Tier 2']
Updated 'Outlet_Location_Type' column with numeric values:
[3 1 2]
Value counts in 'Outlet_Location_Type' column:
Outlet_Location_Type
1    3350
2    2785
3    2388
Name: count, dtype: int64


In [ ]:
df.Outlet_Type = le.fit_transform(df.Outlet_Type)
df['Outlet_Type'] = df['Outlet_Type'].map({0: 1, 1: 2, 2: 3})

In [ ]:
df["Outlet_Type"] = df.groupby(['Outlet_Size','Outlet_Establishment_Year', 'Item_Visibility','Outlet_Location_Type'], as_index=False)['Outlet_Type'].transform(lambda x: x.fillna(x.ffill()))
df.Outlet_Type.fillna("missin").value_counts()
df.Outlet_Type.fillna(df.Outlet_Type.mode()[0],inplace=True)


In [ ]:
year_mapping = {1987: 6, 2004: 6, 1999: 5, 2007: 5, 1985: 4, 1997: 4, 2002: 3, 2009: 2, 1998: 1}
df['Outlet_Establishment_Year'] = df['Outlet_Establishment_Year'].map(year_mapping).fillna(1)

In [ ]:
item_identifier_le = LabelEncoder()
outlet_identifier_le = LabelEncoder()
Item_le = LabelEncoder()
df['Item_Identifier'] = item_identifier_le.fit_transform(df['Item_Identifier'])
df['Item_Type'] = Item_le.fit_transform(df['Item_Type'])


In [ ]:
df['Outlet_Identifier'] = df['Outlet_Identifier'].astype(str).str.extract('(\d+)$')
df.Outlet_Identifier.unique()

array(['049', '018', '010', '013', '027', '045', '017', '046', '035',
       '019'], dtype=object)

In [ ]:
# Calculate mean item weight by Item_Identifier
mean_weights_by_id = df.groupby('Item_Identifier')['Item_Weight'].transform('mean')

# Fill missing Item_Weight values using mean by Item_Identifier
df['Item_Weight'] = df['Item_Weight'].fillna(mean_weights_by_id)

# Calculate median item weight by Item_Type
median_weights_by_type = df.groupby('Item_Type')['Item_Weight'].transform('median')

# Fill remaining missing Item_Weight values using median by Item_Type
df['Item_Weight'] = df['Item_Weight'].fillna(median_weights_by_type)


In [ ]:

# Load data
df1 = pd.read_csv('test.csv')
df2 = df1.copy()

# Replace various cases of 'low fat' and 'reg' in 'Item_Fat_Content'
df1['Item_Fat_Content'] = df1['Item_Fat_Content'].replace(['lf', 'low fat', 'LF'], 'Low Fat')
df1['Item_Fat_Content'] = df1['Item_Fat_Content'].replace('reg', 'Regular')

# Apply Label Encoding to 'Item_Fat_Content'
le = LabelEncoder()
df1['Item_Fat_Content'] = le.fit_transform(df1['Item_Fat_Content'])

# Replace "missing" with NaN
df1 = df1.replace("missing", np.nan)

# Fill missing values in 'Outlet_Size' with "Small"
df1['Outlet_Size'].fillna("Small", inplace=True)

# Map 'Outlet_Size' to numerical values
Sizenum = {
    "High": 3,
    "Medium": 2,
    "Small": 1
}
df1['Outlet_Size'] = df1['Outlet_Size'].map(Sizenum)

# Apply Label Encoding to 'Outlet_Type' and map values
df1['Outlet_Type'] = le.fit_transform(df1['Outlet_Type'])
df1['Outlet_Type'] = df1['Outlet_Type'].map({0: 1, 1: 2, 2: 3})

# Forward fill missing values for 'Outlet_Type' based on other features
df1['Outlet_Type'] = df1.groupby(['Outlet_Size', 'Outlet_Establishment_Year', 'Item_Visibility', 'Outlet_Location_Type'], as_index=False)['Outlet_Type'].transform(lambda x: x.fillna(method='ffill'))

# Fill remaining missing 'Outlet_Type' with the mode
df1['Outlet_Type'].fillna(df1['Outlet_Type'].mode()[0], inplace=True)

# Replace missing 'Outlet_Establishment_Year' values
year_mapping = {1987: 6, 2004: 6, 1999: 5, 2007: 5, 1985: 4, 1997: 4, 2002: 3, 2009: 2, 1998: 1}
df1['Outlet_Establishment_Year'] = df1['Outlet_Establishment_Year'].map(year_mapping).fillna(1)


# Correct mapping dictionary syntax for 'Outlet_Location_Type'
LocationNum = {
    "Tier 1": 3,
    "Tier 2": 2,
    "Tier 3": 1
}

# Replace categorical values with numeric values using the mapping
df1['Outlet_Location_Type'] = df1['Outlet_Location_Type'].map(LocationNum)

# Print the updated column to verify changes
print("Updated 'Outlet_Location_Type' column with numeric values:")
print(df1['Outlet_Location_Type'].unique())


# Replace categorical values with numeric values using the mapping
df1['Outlet_Identifier'] = df1['Outlet_Identifier'].astype(str).str.extract('(\d+)$')

# Apply Label Encoding to 'Item_Identifier', 'Outlet_Identifier', and 'Item_Type'
df1['Item_Identifier'] = le.fit_transform(df1['Item_Identifier'])
df1['Item_Type'] = le.fit_transform(df1['Item_Type'])


# Fill missing 'Item_Weight' values using the mean by 'Item_Identifier'
mean_weights_by_id = df1.groupby('Item_Identifier')['Item_Weight'].transform('mean')
df1['Item_Weight'] = df1['Item_Weight'].fillna(mean_weights_by_id)

# Fill remaining missing 'Item_Weight' values using the median by 'Item_Type'
median_weights_by_type = df1.groupby('Item_Type')['Item_Weight'].transform('median')
df1['Item_Weight'] = df1['Item_Weight'].fillna(median_weights_by_type)
df1.isna().sum()



<ipython-input-12-3ac580684cc8>:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df1['Outlet_Type'] = df1.groupby(['Outlet_Size', 'Outlet_Establishment_Year', 'Item_Visibility', 'Outlet_Location_Type'], as_index=False)['Outlet_Type'].transform(lambda x: x.fillna(method='ffill'))


Updated 'Outlet_Location_Type' column with numeric values:
[3 2 1]


,0
Item_Identifier,0
Item_Weight,0
Item_Fat_Content,0
Item_Visibility,0
Item_Type,0
Item_MRP,0
Outlet_Identifier,0
Outlet_Establishment_Year,0
Outlet_Size,0
Outlet_Location_Type,0


In [ ]:
df1['Outlet_Identifier'] = df1['Outlet_Identifier'].astype(str).str.extract('(\d+)$')
df1.Outlet_Identifier.unique()

array(['049', '017', '010', '027', '046', '018', '045', '019', '013',
       '035'], dtype=object)

In [ ]:
print(df1.select_dtypes(include=[np.number]).columns)


Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Establishment_Year', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')


In [ ]:
features = ['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
            'Item_Type', 'Item_MRP', 'Outlet_Establishment_Year', 'Outlet_Size',
            'Outlet_Location_Type', 'Outlet_Type']
target = 'Item_Outlet_Sales'

# Prepare the training data
train_X = df[features]
train_Y = df[target]

# Prepare the testing data (features only)
test_X = df1[features]


In [ ]:
test_X


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,1103,20.750,0,0.007565,13,107.8622,5,2,3,2.0
1,1067,8.300,1,0.038428,4,87.3198,5,1,2,2.0
2,1406,14.600,0,0.099575,11,241.7538,1,1,1,1.0
3,809,7.315,0,0.015388,13,155.0340,5,1,2,2.0
4,1184,13.600,1,0.118599,4,234.2300,4,2,1,2.0
...,...,...,...,...,...,...,...,...,...,...
5676,231,10.500,1,0.013496,13,141.3154,4,1,3,2.0
5677,306,7.600,1,0.142991,15,169.1448,2,2,1,3.0
5678,1412,10.000,0,0.073529,8,118.7440,3,1,2,2.0
5679,517,15.300,1,0.000000,3,214.6218,5,1,2,2.0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
RF = RandomForestRegressor(n_estimators=100, random_state=42)

RF.fit(train_X,train_Y)

# Fit the model to the training data
RF.fit(train_X, train_Y)

# Make predictions on the test data
predictions = RF.predict(test_X)

# If you want to save the predictions to a CSV file
output = pd.DataFrame({
    'Item_Identifier': df2['Item_Identifier'],
    'Outlet_Identifier': df2['Outlet_Identifier'],  # Include Outlet_Identifier
    'Item_Outlet_Sales': predictions
})
output.to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'")


Predictions saved to 'predictions.csv'


In [ ]:
output

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,1103,049,1661.297502
1,1067,017,1448.068394
2,1406,010,698.643914
3,809,017,1715.793232
4,1184,027,6016.708098
...,...,...,...
5676,231,046,2532.004110
5677,306,018,2335.959300
5678,1412,045,1703.862096
5679,517,017,4737.446636
